In [2]:
import polars as pl

In [4]:
single_parquet_train = pl.read_parquet("/Users/ankushgarg/Downloads/child-mind-institute-problematic-internet-use/series_train.parquet/id=0a418b57/part-0.parquet")
print(single_parquet_train.shape)
single_parquet

(213423, 13)


step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
u32,f32,f32,f32,f32,f32,f32,f32,f32,i64,i8,i8,f32
0,-0.075242,-0.256743,-0.973791,0.038081,-72.952141,0.0,5.0,4202.0,51250000000000,2,4,-9.0
1,-0.265893,-0.270508,-0.76547,0.07743,-52.84922,0.0,0.5,4185.333496,51255000000000,2,4,-9.0
2,0.334517,-0.548602,-0.588596,0.039162,-44.118084,0.0,11.5,4185.5,51260000000000,2,4,-9.0
3,0.000193,-0.021069,-0.999681,0.00145,-88.759613,0.0,0.0,4185.666504,51265000000000,2,4,-9.0
4,-0.000685,-0.020681,-0.997677,0.000491,-88.756958,0.0,8.5,4185.833496,51270000000000,2,4,-9.0
…,…,…,…,…,…,…,…,…,…,…,…,…
213418,0.391111,0.658119,-0.51135,0.060158,-35.455124,0.0,22.0,3817.430664,38745000000000,1,4,25.0
213419,0.551039,0.078267,-0.787324,0.094029,-52.621586,0.0,22.0,3817.430664,38750000000000,1,4,25.0
213420,0.667007,-0.12656,-0.75195,0.013104,-47.961002,0.0,22.0,3817.430664,39015000000000,1,4,25.0


In [16]:
import os
import pandas as pd
import tqdm
from concurrent.futures import ThreadPoolExecutor

def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:

    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = executor.map(lambda fname: process_file(fname, dirname), ids)
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"Stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    
    return df

# df = load_time_series("/Users/ankushgarg/Downloads/child-mind-institute-problematic-internet-use/series_train.parquet")
df = pd.read_parquet("/Users/ankushgarg/Downloads/child-mind-institute-problematic-internet-use/series_train.parquet/id=0a418b57/part-0.parquet")

In [23]:
values_ = df.describe().values.reshape(-1)

In [26]:
pd.DataFrame([values_], columns=[f"Stat_{i}" for i in range(len(values_))])

,Stat_0,Stat_1,Stat_2,Stat_3,Stat_4,Stat_5,Stat_6,Stat_7,Stat_8,Stat_9,...,Stat_94,Stat_95,Stat_96,Stat_97,Stat_98,Stat_99,Stat_100,Stat_101,Stat_102,Stat_103
0,213423.0,213423.0,213423.0,213423.0,213423.0,213423.0,213423.0,213423.0,213423.0,213423.0,...,1.992976,4.674658,89.840576,0.0,2622.600098,4202.0,8.639500e+13,7.0,4.0,25.0


In [29]:
df.describe()

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT
count,213423.000000,213423.000000,213423.000000,213423.000000,213423.000000,213423.000000,213423.0,213423.000000,213423.000000,2.134230e+05,213423.000000,213423.0,213423.000000
mean,106711.000000,0.195960,0.175270,-0.258108,0.056785,-17.940048,0.0,34.272129,3951.493408,5.354966e+13,3.992409,4.0,1.118324
std,61610.057588,0.539906,0.510206,0.453659,0.138671,31.710234,0.0,161.993332,110.120987,1.869022e+13,1.926265,0.0,6.136575
min,0.000000,-2.156349,-2.749820,-1.224756,0.000000,-89.846779,0.0,0.000000,3773.666748,0.000000e+00,1.000000,4.0,-9.000000
25%,53355.500000,-0.215934,-0.144534,-0.609729,0.006549,-39.841597,0.0,2.642857,3847.000000,4.026000e+13,2.000000,4.0,-4.000000
50%,106711.000000,0.304541,0.197180,-0.292209,0.018648,-18.342581,0.0,7.500000,3942.000000,5.448000e+13,4.000000,4.0,1.000000
75%,160066.500000,0.644210,0.586222,0.023711,0.052288,1.038353,0.0,16.416666,4035.000000,6.893000e+13,6.000000,4.0,6.000000
max,213422.000000,1.648549,3.055914,1.992976,4.674658,89.840576,0.0,2622.600098,4202.000000,8.639500e+13,7.000000,4.0,25.000000


In [32]:
df.describe().values.reshape(-1)

array([ 2.13423000e+05,  2.13423000e+05,  2.13423000e+05,  2.13423000e+05,
        2.13423000e+05,  2.13423000e+05,  2.13423000e+05,  2.13423000e+05,
        2.13423000e+05,  2.13423000e+05,  2.13423000e+05,  2.13423000e+05,
        2.13423000e+05,  1.06711000e+05,  1.95960030e-01,  1.75269634e-01,
       -2.58107930e-01,  5.67846261e-02, -1.79400482e+01,  0.00000000e+00,
        3.42721291e+01,  3.95149341e+03,  5.35496633e+13,  3.99240944e+00,
        4.00000000e+00,  1.11832368e+00,  6.16100576e+04,  5.39905608e-01,
        5.10206461e-01,  4.53659356e-01,  1.38670534e-01,  3.17102337e+01,
        0.00000000e+00,  1.61993332e+02,  1.10120987e+02,  1.86902217e+13,
        1.92626505e+00,  0.00000000e+00,  6.13657475e+00,  0.00000000e+00,
       -2.15634942e+00, -2.74982023e+00, -1.22475600e+00,  0.00000000e+00,
       -8.98467789e+01,  0.00000000e+00,  0.00000000e+00,  3.77366675e+03,
        0.00000000e+00,  1.00000000e+00,  4.00000000e+00, -9.00000000e+00,
        5.33555000e+04, -